In [1]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)

In [2]:
def rqst(query):
    number_r = 10
    response = make_a_search_request(query)
    a = response.json()
    while ('results' not in a and number_r > 0):
       print (number_r)
       print ('results' not in a)
       number_r = number_r - 1
       response = make_a_search_request(query)
       a = response.json()
    return a


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
import tensorflow as tf
import sys
import argparse


/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/pyth

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)

In [5]:
saver = tf.train.import_meta_graph('/notebook/uncased_L-12_H-768_A-12/' + 'bert_model.ckpt.meta')
with tf.Session() as sess:
    saver.restore(sess, '/notebook/uncased_L-12_H-768_A-12/' + "bert_model.ckpt")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /notebook/uncased_L-12_H-768_A-12/bert_model.ckpt


In [6]:
import numpy as np

In [7]:
def make_a_search_request(query):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": 10,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

In [8]:
make_a_search_request("Is admission rate in Stanford higher than that of MIT?")

<Response [200]>

In [9]:
a = {}
number = 100
a = make_a_search_request("What is differense between sex and love?")
while ('results' not in in a.json() and number > 0):
    a = make_a_search_request("What is differense between sex and love?")

SyntaxError: invalid syntax (<ipython-input-9-15efa7ceecd7>, line 4)

In [10]:
from xml.dom import minidom

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

In [11]:
!pwd

/notebook/touche


In [12]:
input_dir = '/notebook/touche/'
input_file = 'topics-task-2-only-titles.xml'
list_of_tuples = read_xml(input_dir + input_file)

1
[<DOM Element: topics at 0x7efcac6724b0>]
50


In [23]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
    with open(output_dir + 'run_example.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
        
    for elem in list_of_tuples:
        qid = elem[0]
        print ("qid", qid)
        Q0 = 'Q0'
        query = elem[1]
        tag = 'myBaseline'
        response = make_a_search_request(query)
        a = response.json()
        try:
            scores = [elem['score'] for elem in a['results']]
            docs = [elem['trec_id'] for elem in a['results']]
            titles = [elem['title'] for elem in a['results']]
        except:
            print (a)
        qids = [qid]*len(scores)
        Q0s = [Q0 for elem in scores]
        queries = query*len(scores)
        tags = [tag for elem in scores]
        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list
        
    print ("part 2")    
    for elem in list_of_tuples[:10]:
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'myBertSimilarity'
        #response = make_a_search_request(query)
        #a = response.json()
        #try:
            #docs = [elem['trec_id'] for elem in a['results']]
            #titles = [elem['title'] for elem in a['results']]
            #scores = make_scores_1(query, titles)
        #except:
            #print ("except")
            #return titles
        #qids = qid*len(scores)
        #Q0s = [Q0 for elem in scores]
        #queries = query*len(scores)
        #tags = [tag for elem in scores]
        #part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
        #part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 
        
        #qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)
        
        #ranks = range(1, len(scores) + 1)
        #part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        #common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example1.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [21]:
[1]*5

[1, 1, 1, 1, 1]

In [22]:
run_baseline()

1
[<DOM Element: topics at 0x7efd896585f0>]
50
qid 1
qids ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
part_of_commom_list [('1', 'Q0', 'clueweb12-1214wb-88-29751', 1, 2406.7341, 'myBaseline'), ('1', 'Q0', 'clueweb12-1217wb-96-18723', 2, 2396.9736, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08418', 3, 2396.6697, 'myBaseline'), ('1', 'Q0', 'clueweb12-1809wb-33-13145', 4, 2396.6697, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08424', 5, 2396.6616, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08423', 6, 2396.4927, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08421', 7, 2395.756, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08425', 8, 2395.4934, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08419', 9, 2394.7332, 'myBaseline'), ('1', 'Q0', 'clueweb12-1811wb-62-08422', 10, 2394.7102, 'myBaseline')]
qid 2
qids ['2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
part_of_commom_list [('2', 'Q0', 'clueweb12-0505wb-78-06026', 1, 2183.8928, 'myBaseline'), ('2', 'Q0', 'clue

In [8]:
def make_scores_0(query, answer_scores):
    # take answer_scores from clueweb as scores 
    # retrun the same scores (baseline)
    return answer_scores

In [13]:
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
bc = BertClient()

import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

def make_scores_1(query, answer_titles):
    query_emb = bc.encode([query])
    scores = [cosine_similarity(query_emb.reshape(1, -1), bc.encode([cleanhtml(answer_title)]).reshape(1, -1))[0][0] for answer_title in answer_titles]
    return scores
    
    
    # take title from seacrh output from clueweb
    # return a cosine similarity between bert embedding of sentences and article title

In [35]:
import tensorflow as tf
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('/notebook/uncased_L-12_H-768_A-12/bert_model.ckpt.meta')
    saver.restore(sess, "/notebook/uncased_L-12_H-768_A-12/bert_model.ckpt")

INFO:tensorflow:Restoring parameters from /notebook/uncased_L-12_H-768_A-12/bert_model.ckpt


In [36]:
bc = BertClient()
encode1 = bc.encode(['What is better bread or pizza'])
len(encode1[0])

768

In [5]:
response = make_a_search_request('What is better bread or pizza')

In [6]:
a = response.json()


In [8]:
len(a['results'])

1000

In [24]:
a['results'][1]['snippet']

'We did decide it would be worth it to invest in some <em>better</em> <em>bread</em> pans. The kind most people have at home (and <em>what</em> I had) are actually smaller than a “real” <em>bread</em> loaf – great for banana <em>bread</em>, but not so much for regular <em>bread</em>-making. I chose a pair of 1 1&#x2F;2 lb.'

In [25]:
a['results'][1]['score']

1665.953

In [ ]:
qid Q0 doc rank score tag
1 Q0 clueweb09-en0010-85-29836 1 17.89 myGroupMyMethod

In [2]:
Q0 = 'Q0'
Q0*10

'Q0Q0Q0Q0Q0Q0Q0Q0Q0Q0'

In [16]:
list_of_tuples = read_xml('topics-task-2-only-titles.xml')
common_list = []
for elem in list_of_tuples:
    qid = elem[0]
    Q0 = 'Q0'
    query = elem[1]
    tag = 'myBaseline'
    print ("query", query)
    response = make_a_search_request(query)
    a = response.json()
    scores = [elem['score'] for elem in a['results']]
    docs = [elem['trec_id'] for elem in a['results']]
    titles = [elem['title'] for elem in a['results']]
    qids = qid*len(scores)
    Q0s = [Q0 for elem in scores]
    queries = query*len(scores)
    tags = [tag for elem in scores]
    ranks = range(1, len(scores) + 1)
    part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    print ("len of recieve part, len of common list", len(part_of_commom_list), len(common_list))
    common_list = common_list + part_of_commom_list
    
with open('/notebook/touche/output/run_example.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

1
[<DOM Element: topics at 0x7f8973bc8eb0>]
50
query What is the difference between sex and love?
len of recieve part, len of common list 10 0
query Which is better, laptop or desktop?
len of recieve part, len of common list 10 10
query Which is better, Canon or Nikon?
len of recieve part, len of common list 10 20
query What are the best dish detergents?
len of recieve part, len of common list 10 30
query What are the best cities to live?
len of recieve part, len of common list 10 40
query What is the longest river in the U.S.?
len of recieve part, len of common list 10 50
query Which is healthiest: coffee, green tea or black tea and why?
len of recieve part, len of common list 10 60
query What are advantages and disadvantages of PHP over Python and vice versa?
len of recieve part, len of common list 10 70
query Why is Linux better than Windows?
len of recieve part, len of common list 10 80
query How to sleep better?
len of recieve part, len of common list 10 90
query Should I buy an L

In [22]:
with open('/notebook/touche/output/run_example.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [145]:
a = list(range(1, 17))
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [17]:
!pwd

/notebook/touche


In [ ]:
title, snippet, score, trec_id

In [140]:
scores = [elem['score'] for elem in a['results']]
docs = [elem['trec_id'] for elem in a['results']]
titles = [elem['title'] for elem in a['results']]

In [141]:
scores

[1763.1145,
 1665.953,
 1665.794,
 1607.0322,
 1464.4408,
 1371.346,
 1363.5419,
 1348.255,
 1314.545,
 1314.2784]

In [142]:
a

{'meta': {'query_time': 3348, 'total_results': 63265, 'indices': ['cw12']},
 'results': [{'score': 1763.1145,
   'uuid': 'fdb00460-5461-5440-8d51-48b5715ddeef',
   'index': 'cw12',
   'trec_id': 'clueweb12-0712wb-94-14296',
   'target_hostname': 'freshandfoodie.com',
   'target_uri': 'http://freshandfoodie.com/category/breads-pizzas/',
   'page_rank': 2.6386606e-09,
   'spam_rank': 83,
   'title': '<em>Breads</em> &amp; <em>Pizzas</em> «',
   'snippet': 'The perfect flavors, plus the pretzel nugget on top <em>is</em> super cute. Yep, they went there. There are Oreos baked into these brownies. So, which are you? A <em>better</em> cook <em>or</em> a <em>better</em> baker? <em>What</em> have you seen on the interwebs lately that inspired you? Please share!',
   'explanation': None},
  {'score': 1665.953,
   'uuid': '282104d4-6839-5b4f-afe9-5b3759e7a20a',
   'index': 'cw12',
   'trec_id': 'clueweb12-1606wb-34-03244',
   'target_hostname': 'www.as-for-my-house.com',
   'target_uri': 'http:/